In [2]:
import pytz
import requests
import json
from datetime import datetime, time
import time as time_module 


def Mode(state):
  new_state=state
  # Credentials and IDs
  client_id = "VxOyk2wQ9L0SVPijTZi2WEh6MA4FAbVo"
  client_secret="PznqJ4ZwlKQ5pBBkqPM00a1k!G3XXjElYP1RbGtZz4NKVwJgmXK@bK!2bZBDdxai"
  thing_id = "6a45bf50-3f2f-461a-9e37-18cfc466496a"
  property_id = "abd9eedc-dc74-47de-930b-a9dda7dda31f"

  # Function to get access token
  def get_access_token(client_id, client_secret):
      url = "https://api2.arduino.cc/iot/v1/clients/token"
      data = {
          'grant_type': 'client_credentials',
          'client_id': client_id,
          'client_secret': client_secret,
          'audience': 'https://api2.arduino.cc/iot'
      }
      response = requests.post(url, data=data)
      if response.status_code == 200:
          return response.json()['access_token']
      else:
          print(f"Error getting access token: {response.status_code} {response.text}")
          return None

  # Function to update and publish the property
  def update_and_publish_property(access_token, thing_id, property_id, new_state):
      headers = {
          "Accept" : "application/vnd.arduino.property+json,application/vnd.goa.error+json",
          'Authorization': f'Bearer {access_token}',
          'Content-Type': 'application/json'
      }

      # Update the property value
      update_url = f"https://api2.arduino.cc/iot/v2/things/{thing_id}/properties/{property_id}"
      update_payload = {'value': new_state}
      update_response = requests.put(update_url, json=update_payload, headers=headers)

      # Check update response
      if update_response.status_code == 200:
          try:
              update_result = update_response.json()
          except json.JSONDecodeError:
              update_result = f"Update successful, but no JSON returned: {update_response.status_code}"
      else:
          update_result = f"Update error: {update_response.status_code} {update_response.text}"

      # Publish the property value
      publish_url = f"https://api2.arduino.cc/iot/v2/things/{thing_id}/properties/{property_id}/publish"
      publish_response = requests.put(publish_url, json=update_payload, headers=headers)

      # Check publish response
      if publish_response.status_code == 200:
          try:
              publish_result = publish_response.json()
          except json.JSONDecodeError:
              publish_result = f"Publish successful, but no JSON returned: {publish_response.status_code}"
      else:
          publish_result = f"Publish error: {publish_response.status_code} {publish_response.text}"

      return update_result, publish_result

  # Main execution
  access_token = get_access_token(client_id, client_secret)
  if access_token:
      update_result, publish_result = update_and_publish_property(access_token, thing_id, property_id, new_state)
      print(f"Update Result: {update_result}")
      print(f"Publish Result: {publish_result}")



# Get the current date and time in the 'Asia/Kolkata' timezone using pytz
kolkata_tz = pytz.timezone('Asia/Kolkata')

# Define the target time
target_time_ON = time(17, 30,30)
target_time_OFF = time(20, 30,30)
while True:
      while True:
          # Get the current date and time in the 'Asia/Kolkata' timezone
          dtobj = datetime.now(kolkata_tz)
      
          # Check if the current time is greater than or equal to the target time
          if (dtobj.time() >= target_time_ON and dtobj.time() <= target_time_OFF) :
              Mode(True)
              break  # Exit the loop
      
          # Delay for a while (e.g., 1 second) before checking again
          time_module.sleep(1)
      while True:
        # Get the current date and time in the 'Asia/Kolkata' timezone
        dtobj = datetime.now(kolkata_tz)
      
        # Check if the current time is greater than or equal to the target time
        if dtobj.time() >= target_time_OFF:
            Mode(False)
            break  # Exit the loop
      
        # Delay for a while (e.g., 1 second) before checking again
        time_module.sleep(1)

Update Result: Update error: 405 {"id":"XnorRFTb","code":"method_not_allowed","status":405,"detail":"Method PUT must be OPTIONS","meta":{"allowed":"OPTIONS","method":"PUT"}}

Publish Result: Publish successful, but no JSON returned: 200


KeyboardInterrupt: 